# ДЗ 07. ML для бизнеса

Домашнее задание (часть 1)

1. [взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)](#task01)
2. [сделать eda (как на занятии 6 с примером для оттока)](#task02)
* [сделать feature engineering](#task02)
* [обучить любой классификатор (какой вам нравится)](#task04)
* [далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все * положительные (класс 1) примеры, а только лишь часть](#task05)
* [применить random negative sampling для построения классификатора в новых условиях](#task06)
* [сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)](#task07)

Домашнее задание (часть 2)

Придумать новые признаки (у нас их всего 5 было) - сделать дополнительный feature engineering
Для подходов с одной моделью поварьировать параметры CatBoostClassifier либо даже попробовать другой алгоритм (опционально уже).
Для пункта 2 (подходов на основе одной модели) вывести значимость признаков, основываясь на model.estimator.feature_importances_ и model.estimator.feature_names_.
Дополнить models_results дополнительными метриками (roc auc, logloss, f1). В конце вывести сводную таблицу всех показателей для всех подходов (как на основе


In [380]:
import pandas as pd
import numpy as np

# Input data files are available in the "../input/" directory.
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization

np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

## Задание 1

взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
сделать eda (как на занятии 6 с примером для оттока) <a class='anchor' id='task01'></a>

Нашли и скачали файл по ссылке: https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip, разархивировали, достали файл положили в ту же папку, что и скрипт

In [381]:
PATH = "bank.csv"
data = pd.read_csv(PATH, sep=';')

In [382]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


## Задание 2-3. EDA + Feature Engineering <a class="anchor" id="task02"></a>

Проверим сбалансированность классов:

In [383]:
data.iloc[:,-1].value_counts()

no     4000
yes     521
Name: y, dtype: int64

Общий взгляд на датасет:

In [384]:
print ("Rows     : " ,data.shape[0])
print ("Columns  : " ,data.shape[1])
print ("\nFeatures : \n" ,data.columns.tolist())
print ("\nMissing values :  ", data.isnull().sum().values.sum()) #telcom.isnull().sum().values.sum()
print ("\nUnique values :  \n",data.nunique())

Rows     :  4521
Columns  :  17

Features : 
 ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

Missing values :   0

Unique values :  
 age            67
job            12
marital         3
education       4
default         2
balance      2353
housing         2
loan            2
contact         3
day            31
month          12
duration      875
campaign       32
pdays         292
previous       24
poutcome        4
y               2
dtype: int64


Проверка на пустые ячейки, пробелы в строковых данных

In [385]:
all_cols = data.columns.tolist() # все столбцы сохраняем в список
for column in all_cols:
    indexes = data[data[column]==' '].index.tolist() # список индексов ячеек с пробелами
    indexes1 = data[data[column]==''].index.tolist()
    if (len(indexes)>0) or (len(indexes1)>0):
        item = {'column':column, 'indexes':indexes+indexes1}
        print(item)

Пустых ячеек и пробелов нет! Проверим формат данных:

In [386]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


Графическое отображение несбалансированности классов:

In [387]:
#labels
lab = data["y"].value_counts().keys().tolist()
#values
val = data["y"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )
layout = go.Layout(dict(
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data1 = [trace]
fig = go.Figure(data = data1,layout = layout)
py.iplot(fig)

### Data manipulation

#### Разделим датасет на две части по условию принадлежности к меткам классов

In [388]:
data_1=data[data['y']=='yes']
data_0=data[data['y']=='no']

#### Разделение на категориальные и количественные столбцы

In [389]:
target_col = ["y"]
cat_cols   = data.nunique()[data.nunique() < 13].keys().tolist() # вывести все признаки(столбцы), в котором меньше 13 категорий

In [390]:
cat_cols   = [x for x in cat_cols if x not in target_col] # убираем столбец с таргетом из категориальных признаков
num_cols   = [x for x in data.columns if x not in cat_cols + target_col] #  все остальные столбцы (признаки) относим к количественным

In [391]:
cat_cols, num_cols

(['job',
  'marital',
  'education',
  'default',
  'housing',
  'loan',
  'contact',
  'month',
  'poutcome'],
 ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'])

In [392]:
#function  for pie plot for customer attrition types
def plot_pie(data_1, data_0, column) :
    
    trace1 = go.Pie(values  = data_1[column].value_counts().values.tolist(),
                    labels  = data_1[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "target 1",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = data_0[column].value_counts().values.tolist(),
                    labels  = data_0[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "target 0" 
                   )


    layout = go.Layout(dict(title = column + " distribution",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "target 1(yes)",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "target 0 (no)",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data = [trace1,trace2]
    fig  = go.Figure(data = data,layout = layout)
    py.iplot(fig)


#function  for histogram for customer attrition types
def histogram(data_1, data_0, column) :
    trace1 = go.Histogram(x  = data_1[column],
                          histnorm= "percent",
                          name = "target 1",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = .9 
                         ) 
    
    trace2 = go.Histogram(x  = data_0[column],
                          histnorm = "percent",
                          name = "target 0 customers",
                          marker = dict(line = dict(width = .5,
                                              color = "black"
                                             )
                                 ),
                          opacity = .9
                         )
    
    data = [trace1,trace2]
    layout = go.Layout(dict(title =column + " distribution ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = column,
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "percent",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                           )
                      )
    fig  = go.Figure(data=data,layout=layout)
    
    py.iplot(fig)
    
#function  for scatter plot matrix  for numerical columns in data
def scatter_matrix(df):
    
    df  = df.sort_values(by = "y" ,ascending = True)
    classes = df["y"].unique().tolist()
    classes
    
    class_code  = {classes[k] : k for k in range(2)}
    class_code

    color_vals = [class_code[cl] for cl in df["y"]]
    color_vals

    pl_colorscale = "Portland"

    pl_colorscale

    text = [df.loc[k,"y"] for k in range(len(df))]
    text

    trace = go.Splom(dimensions = [dict(label  = "age",
                                       values = df["age"]),
                                  dict(label  = 'balance',
                                       values = df['balance']),
                                  dict(label  = 'duration',
                                       values = df['duration'])],
                     text = text,
                     marker = dict(color = color_vals,
                                   colorscale = pl_colorscale,
                                   size = 3,
                                   showscale = False,
                                   line = dict(width = .1,
                                               color='rgb(230,230,230)'
                                              )
                                  )
                    )
    axis = dict(showline  = True,
                zeroline  = False,
                gridcolor = "#fff",
                ticklen   = 4
               )
    
    layout = go.Layout(dict(title  = 
                            "Scatter plot matrix for Numerical columns for customer attrition",
                            autosize = False,
                            height = 800,
                            width  = 800,
                            dragmode = "select",
                            hovermode = "closest",
                            plot_bgcolor  = 'rgba(240,240,240, 0.95)',
                            xaxis1 = dict(axis),
                            yaxis1 = dict(axis),
                            xaxis2 = dict(axis),
                            yaxis2 = dict(axis),
                            xaxis3 = dict(axis),
                            yaxis3 = dict(axis),
                           )
                      )
    data   = [trace]
    fig = go.Figure(data = data,layout = layout )
    py.iplot(fig)

In [393]:
#for all categorical columns plot pie
for i in cat_cols :
    plot_pie(data_1, data_0, i)

#for all categorical columns plot histogram    
for i in num_cols :
    histogram(data_1, data_0, i)
    
# #scatter plot matrix
scatter_matrix(data)

На первый взгляд, глядя на графики признаков, можно сказать, что почти все они (за исключением 'Default') влияют на распределение целевой переменной

### Data Preprocessing

In [394]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

#### Определим столбцы с бинарными данными и где значений признака больше двух

In [395]:
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist() # определяем столбцы, где только два значения признака
#Columns more than 2 values
multi_cols = [i for i in cat_cols if i not in bin_cols] # определяем столбцы, где значений больше, чем два

In [396]:
bin_cols, multi_cols

(['default', 'housing', 'loan', 'y'],
 ['job', 'marital', 'education', 'contact', 'month', 'poutcome'])

#### Кодировка столбцов с бинарными признаками

In [397]:
#Label encoding Binary columns
le = LabelEncoder()
for i in bin_cols :
    data[i] = le.fit_transform(data[i])

#### Столбец с месяцами превращаем в цифровой

In [398]:
data['month'].replace({'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10,
                      'nov':11, 'dec':12}, inplace=True)


In [399]:
# исключаем этот признак из категориальных, и засовываем его в цифровой
cat_cols = [name for name in cat_cols if name!='month']
multi_cols = [name for name in multi_cols if name!='month']
num_cols.append('month')

In [400]:
cat_cols, num_cols, multi_cols

(['job',
  'marital',
  'education',
  'default',
  'housing',
  'loan',
  'contact',
  'poutcome'],
 ['age',
  'balance',
  'day',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'month'],
 ['job', 'marital', 'education', 'contact', 'poutcome'])

In [401]:
data['month']=data['month'].astype('int64')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,10,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,5,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,4,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,6,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,5,226,1,-1,0,unknown,0


#### Столбец с месяцами превращаем в цифровой

In [402]:
data['education'].unique()

array(['primary', 'secondary', 'tertiary', 'unknown'], dtype=object)

In [403]:
data['education'].replace({'primary':1, 'secondary':2, 'tertiary':3, 'unknown':2}, inplace=True)


In [404]:
data['education'] = data['education'].astype('int64')
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,0,0,cellular,19,10,79,1,-1,0,unknown,0
1,33,services,married,2,0,4789,1,1,cellular,11,5,220,1,339,4,failure,0
2,35,management,single,3,0,1350,1,0,cellular,16,4,185,1,330,1,failure,0
3,30,management,married,3,0,1476,1,1,unknown,3,6,199,4,-1,0,unknown,0
4,59,blue-collar,married,2,0,0,1,0,unknown,5,5,226,1,-1,0,unknown,0


In [405]:
# исключаем этот признак из категориальных, и засовываем его в цифровой
cat_cols = [name for name in cat_cols if name!='education']
multi_cols = [name for name in multi_cols if name!='education']
num_cols.append('education')

In [406]:
cat_cols, multi_cols, num_cols

(['job', 'marital', 'default', 'housing', 'loan', 'contact', 'poutcome'],
 ['job', 'marital', 'contact', 'poutcome'],
 ['age',
  'balance',
  'day',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'month',
  'education'])

In [407]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,1,0,1787,0,0,cellular,19,10,79,1,-1,0,unknown,0
1,33,services,married,2,0,4789,1,1,cellular,11,5,220,1,339,4,failure,0
2,35,management,single,3,0,1350,1,0,cellular,16,4,185,1,330,1,failure,0
3,30,management,married,3,0,1476,1,1,unknown,3,6,199,4,-1,0,unknown,0
4,59,blue-collar,married,2,0,0,1,0,unknown,5,5,226,1,-1,0,unknown,0


#### marital status обработка, превращение в бинарный признак

In [408]:
data.loc[data['marital']=='married', ['marital']]=1
data.loc[(data['marital']=='single')|(data['marital']=='divorced'), ['marital']]=0

In [411]:
data['marital']=data['marital'].astype('int64')

In [412]:
# исключаем этот признак из категориальных, и засовываем его в цифровой
cat_cols = [name for name in cat_cols if name!='marital']
multi_cols = [name for name in multi_cols if name!='marital']
num_cols.append('marital')

In [414]:
cat_cols, num_cols, multi_cols

(['job', 'default', 'housing', 'loan', 'contact', 'poutcome'],
 ['age',
  'balance',
  'day',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'month',
  'education',
  'marital'],
 ['job', 'contact', 'poutcome'])

#### contact обработка, превращение в бинарный признак

In [416]:
data.loc[data['contact']=='unknown', ['contact']]=1
data.loc[(data['contact']=='cellular')|(data['contact']=='telephone'), ['contact']]=0

In [417]:
data['contact']=data['contact'].astype('int64')

In [425]:
# исключаем этот признак из категориальных, и засовываем его в цифровой
cat_cols = [name for name in cat_cols if name!='contact']
multi_cols = [name for name in multi_cols if name!='contact']
num_cols.append('contact')

#### poutcome обработка, превращение в бинарный признак

In [420]:
data.loc[data['poutcome']=='unknown', ['poutcome']]=1

In [423]:
data.loc[data['poutcome']!=1, ['poutcome']]=0
data['poutcome'].unique()
data['poutcome']=data['poutcome'].astype('int64')

In [426]:
# исключаем этот признак из категориальных, и засовываем его в цифровой
cat_cols = [name for name in cat_cols if name!='poutcome']
multi_cols = [name for name in multi_cols if name!='poutcome']
num_cols.append('poutcome')

In [427]:
cat_cols, num_cols, multi_cols

(['job', 'default', 'housing', 'loan'],
 ['age',
  'balance',
  'day',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'month',
  'education',
  'marital',
  'contact',
  'poutcome'],
 ['job'])

In [428]:
['default', 'housing', 'loan']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   int64 
 3   education  4521 non-null   int64 
 4   default    4521 non-null   int64 
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   int64 
 7   loan       4521 non-null   int64 
 8   contact    4521 non-null   int64 
 9   day        4521 non-null   int64 
 10  month      4521 non-null   int64 
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   int64 
 16  y          4521 non-null   int64 
dtypes: int64(16), object(1)
memory usage: 600.6+ KB


#### Преобразование столбца 'job'. Выясним, какие значения признака увеличивают вероятность таргета, а какие нет

In [473]:
# проверим связь признака 'job' с целевой переменной
job_column = pd.DataFrame(data_1['job'].value_counts(), columns=['job'])
job_column['%_0']=(job_column['job']/job_column['job'].sum())
job_column['1_qty']=(data_0['job'].value_counts())
job_column['%_1']=(job_column['1_qty']/job_column['1_qty'].sum())
job_column['influence']=0
job_column.loc[(job_column['%_1']/job_column['%_0'])>1, ['influence']]=1

job_column

,job,%_0,1_qty,%_1,influence
management,131,0.251440,838,0.20950,0
technician,83,0.159309,685,0.17125,1
blue-collar,69,0.132438,877,0.21925,1
admin.,58,0.111324,420,0.10500,0
retired,54,0.103647,176,0.04400,0
services,38,0.072937,379,0.09475,1
self-employed,20,0.038388,163,0.04075,1
student,19,0.036468,65,0.01625,0
entrepreneur,15,0.028791,153,0.03825,1
housemaid,14,0.026871,98,0.02450,0


In [474]:
job_1 = job_column.loc[job_column['influence']==1].index.tolist()
job_1

['technician',
 'blue-collar',
 'services',
 'self-employed',
 'entrepreneur',
 'unemployed']

In [477]:
# data.loc[data['job'] in job_1]
data.loc[data['job'].isin(job_1), ['job']]=1

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,1,1,1,0,1787,0,0,0,19,10,79,1,-1,0,1,0
1,33,1,1,2,0,4789,1,1,0,11,5,220,1,339,4,0,0
2,35,management,0,3,0,1350,1,0,0,16,4,185,1,330,1,0,0
3,30,management,1,3,0,1476,1,1,1,3,6,199,4,-1,0,1,0
4,59,1,1,2,0,0,1,0,1,5,5,226,1,-1,0,1,0
5,35,management,0,3,0,747,0,0,0,23,2,141,2,176,3,0,0
6,36,1,1,3,0,307,1,0,0,14,5,341,1,330,2,0,0
7,39,1,1,2,0,147,1,0,0,6,5,151,2,-1,0,1,0
8,41,1,1,3,0,221,1,0,1,14,5,57,2,-1,0,1,0
9,43,1,1,1,0,-88,1,1,0,17,4,313,1,147,2,0,0


In [478]:
data.loc[data['job']!=1, ['job']]=0

In [480]:
data['job']=data['job'].astype('int64')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   age        4521 non-null   int64
 1   job        4521 non-null   int64
 2   marital    4521 non-null   int64
 3   education  4521 non-null   int64
 4   default    4521 non-null   int64
 5   balance    4521 non-null   int64
 6   housing    4521 non-null   int64
 7   loan       4521 non-null   int64
 8   contact    4521 non-null   int64
 9   day        4521 non-null   int64
 10  month      4521 non-null   int64
 11  duration   4521 non-null   int64
 12  campaign   4521 non-null   int64
 13  pdays      4521 non-null   int64
 14  previous   4521 non-null   int64
 15  poutcome   4521 non-null   int64
 16  y          4521 non-null   int64
dtypes: int64(17)
memory usage: 600.6 KB


#### Из категориальных столбцов, содержащих множественные (больше двух) значений создаём бинарные признаки (новые столбцы)

In [235]:
multi_cols

['job', 'marital', 'education', 'contact', 'poutcome']

In [236]:
#Duplicating columns for multi value columns
datab = data.copy()
data = pd.get_dummies(data = data,columns = multi_cols )

#### Стандартизируем данные и вставляем в датасет

In [237]:
data

,age,default,balance,housing,loan,day,month,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_divorced,marital_married,marital_single,education_1,education_2,education_3,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,0,1787,0,0,19,10,79,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1
1,33,0,4789,1,1,11,5,220,1,339,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0
2,35,0,1350,1,0,16,4,185,1,330,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0
3,30,0,1476,1,1,3,6,199,4,-1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
4,59,0,0,1,0,5,5,226,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,0,-333,1,0,30,7,329,5,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1
4517,57,1,-3313,1,1,9,5,153,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
4518,57,0,295,0,0,19,8,151,11,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1
4519,28,0,1137,0,0,6,2,129,4,211,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0


In [114]:
#Scaling Numerical columns
std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

KeyError: "['month'] not in index"

In [20]:
#dropping original values merging scaled values for numerical columns
df_og = data.copy()
data = data.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index=True,right_index=True,how = "left")

In [21]:
summary = (df_og[[i for i in df_og.columns]].
           describe().transpose().reset_index())

summary = summary.rename(columns = {"index" : "feature"})
summary = np.around(summary,3)

val_lst = [summary['feature'], summary['count'],
           summary['mean'],summary['std'],
           summary['min'], summary['25%'],
           summary['50%'], summary['75%'], summary['max']]

trace  = go.Table(header = dict(values = summary.columns.tolist(),
                                line = dict(color = ['#506784']),
                                fill = dict(color = ['#119DFF']),
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["lightgrey",'#F5F8FF'])
                               ),
                  columnwidth = [200,60,100,100,60,60,80,80,80])
layout = go.Layout(dict(title = "Variable Summary"))
figure = go.Figure(data=[trace],layout=layout)
py.iplot(figure)

In [22]:
data

,default,housing,loan,y,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,...,poutcome_other,poutcome_success,poutcome_unknown,age,balance,day,duration,campaign,pdays,previous
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,-1.056270,0.121072,0.374052,-0.711861,-0.576829,-0.407218,-0.320413
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,-0.772583,1.118644,-0.596026,-0.169194,-0.576829,2.989044,2.041734
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,-0.583458,-0.024144,0.010273,-0.303898,-0.576829,2.899143,0.270124
3,0,1,1,0,0,0,0,0,1,0,...,0,0,1,-1.056270,0.017726,-1.566105,-0.250017,0.387967,-0.407218,-0.320413
4,0,1,0,0,0,1,0,0,0,0,...,0,0,1,1.686036,-0.472753,-1.323585,-0.146102,-0.576829,-0.407218,-0.320413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,0,1,0,0,0,0,0,0,0,0,...,0,0,1,-0.772583,-0.583410,1.707910,0.250315,0.709566,-0.407218,-0.320413
4517,1,1,1,0,0,0,0,0,0,0,...,0,0,1,1.496912,-1.573671,-0.838546,-0.427057,-0.576829,-0.407218,-0.320413
4518,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1.496912,-0.374724,0.374052,-0.434754,2.639160,-0.407218,-0.320413
4519,0,0,0,0,0,1,0,0,0,0,...,1,0,0,-1.245394,-0.094925,-1.202326,-0.519426,0.387967,1.710451,1.451197


## Задание 4. Моделирование <a class='anchor' id ='task04'></a>

In [23]:
from sklearn.model_selection import train_test_split
y_data = data['y']
x_data = data.copy()
x_data = x_data.drop(columns=['y'])

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [37]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    #print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    f1_w = f1_score(y_test, y_predict, average='weighted')
    #print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    #print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    #print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    #print("precision: %.2f%%" % (prc * 100.0)) 
    
    item = {'f1':f1, 'f1_weighted':f1_w, 'roc':roc, 'rec':rec,'prc':prc}
    
    return item


In [38]:
%%time
import xgboost as xgb
n_trees = [5,25,50,100,500,1000]
depths = [2,5,10,20]
test_results = []

for trees in n_trees:
    for depth in depths:
        model = xgb.XGBClassifier(n_trees=trees, max_depth=depth)

        model.fit(x_train, y_train, )
        y_predict = model.predict(x_test)
        
        evals = evaluate_results(y_test, y_predict)
        evals['trees']=trees
        evals['depth']=depth
        print(evals)
        test_results.append(evals)
        

{'f1': 0.380952380952381, 'f1_weighted': 0.8804287506705755, 'roc': 0.6252520887352347, 'rec': 0.2692307692307692, 'prc': 0.6511627906976745, 'trees': 5, 'depth': 2}
{'f1': 0.3684210526315789, 'f1_weighted': 0.8761735026207228, 'roc': 0.6221309901085182, 'rec': 0.2692307692307692, 'prc': 0.5833333333333334, 'trees': 5, 'depth': 5}
{'f1': 0.41509433962264153, 'f1_weighted': 0.882928048623179, 'roc': 0.644921012196293, 'rec': 0.3173076923076923, 'prc': 0.6, 'trees': 5, 'depth': 10}
{'f1': 0.4125, 'f1_weighted': 0.8820632847815169, 'roc': 0.6442967924709498, 'rec': 0.3173076923076923, 'prc': 0.5892857142857143, 'trees': 5, 'depth': 20}
{'f1': 0.380952380952381, 'f1_weighted': 0.8804287506705755, 'roc': 0.6252520887352347, 'rec': 0.2692307692307692, 'prc': 0.6511627906976745, 'trees': 25, 'depth': 2}
{'f1': 0.3684210526315789, 'f1_weighted': 0.8761735026207228, 'roc': 0.6221309901085182, 'rec': 0.2692307692307692, 'prc': 0.5833333333333334, 'trees': 25, 'depth': 5}
{'f1': 0.415094339622641

In [39]:
res_df = pd.DataFrame(test_results)
(res_df.sort_values('f1',ascending=False)).head(10)

,f1,f1_weighted,roc,rec,prc,trees,depth
14,0.415094,0.882928,0.644921,0.317308,0.600000,100,10
2,0.415094,0.882928,0.644921,0.317308,0.600000,5,10
22,0.415094,0.882928,0.644921,0.317308,0.600000,1000,10
6,0.415094,0.882928,0.644921,0.317308,0.600000,25,10
18,0.415094,0.882928,0.644921,0.317308,0.600000,500,10
10,0.415094,0.882928,0.644921,0.317308,0.600000,50,10
23,0.412500,0.882063,0.644297,0.317308,0.589286,1000,20
3,0.412500,0.882063,0.644297,0.317308,0.589286,5,20
7,0.412500,0.882063,0.644297,0.317308,0.589286,25,20
19,0.412500,0.882063,0.644297,0.317308,0.589286,500,20


Результаты не очень. Почти на грани. Если обмануть оценку, подставив 'weighted', то можно получить что-то более-менее внятное (поле 'f1_weghted'). Что, если бы мы взяли рандомное предсказание:

In [76]:
# Создадим массив, в котором случайно проставим 10 единичек
from random import randint
for x in range(10):
    g = randint(0, 904)
    y_zeroes[g]=1

In [77]:
evaluate_results(y_test, y_zeroes)

{'f1': 0.07582938388625592,
 'f1_weighted': 0.7858600321490634,
 'roc': 0.4766637856525497,
 'rec': 0.07692307692307693,
 'prc': 0.07476635514018691}

### Попробуем сбалансировать выборку методом oversampling

### SMOTE

- Randomly pick a point from the minority class.
- Compute the k-nearest neighbors (for some pre-specified k) for this point.
- Add k new points somewhere between the chosen point and each of its neighbors

In [81]:
from imblearn.over_sampling import SMOTE

cols    = [i for i in data.columns if i not in target_col]
cols

['default',
 'housing',
 'loan',
 'job_admin.',
 'job_blue-collar',
 'job_entrepreneur',
 'job_housemaid',
 'job_management',
 'job_retired',
 'job_self-employed',
 'job_services',
 'job_student',
 'job_technician',
 'job_unemployed',
 'job_unknown',
 'marital_divorced',
 'marital_married',
 'marital_single',
 'education_primary',
 'education_secondary',
 'education_tertiary',
 'education_unknown',
 'contact_cellular',
 'contact_telephone',
 'contact_unknown',
 'month_apr',
 'month_aug',
 'month_dec',
 'month_feb',
 'month_jan',
 'month_jul',
 'month_jun',
 'month_mar',
 'month_may',
 'month_nov',
 'month_oct',
 'month_sep',
 'poutcome_failure',
 'poutcome_other',
 'poutcome_success',
 'poutcome_unknown',
 'age',
 'balance',
 'day',
 'duration',
 'campaign',
 'pdays',
 'previous']

In [ ]:


smote_X = data[cols]
smote_Y = data[target_col]

#Split train and test data
smote_train_X,smote_test_X,smote_train_Y,smote_test_Y = train_test_split(smote_X,smote_Y,
                                                                         test_size = .25 ,
                                                                         random_state = 111)

#oversampling minority class using smote
os = SMOTE(random_state = 0)
os_smote_X,os_smote_Y = os.fit_sample(smote_train_X,smote_train_Y)
os_smote_X = pd.DataFrame(data = os_smote_X,columns=cols)
os_smote_Y = pd.DataFrame(data = os_smote_Y,columns=target_col)
###



logit_smote = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

telecom_churn_prediction(logit_smote,os_smote_X,test_X,os_smote_Y,test_Y,
                         cols,"coefficients",threshold_plot = True)

## Задание 5 <a class='anchor' id='task05'></a>

разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все * положительные (класс 1) примеры, а только лишь часть

In [50]:
y_0 = np.full(y_test.shape, 0)

In [47]:
y_zeroes = np.zeros(y_test.shape[0])